In [1]:
import pandas as pd
from pymongo import MongoClient
import re
import folium
import json
from folium.plugins import MarkerCluster
import requests
client = MongoClient()
import function as f

In [2]:
punt=pd.read_csv('../OUTPUT/master99.csv')

In [5]:
db, coll = f.connectCollection('companies','preferred')

In [6]:
business = list(coll.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

All collections should be find as json Arrays in OUTPUT's folder

In [7]:
#mongoexport -dcompanies -c schools -o schools.json --jsonArray

In [8]:
empl={'num':[20,5,10,15,5,20,1,10,1,26,87], 
      'wot':['Designers','UI/UX Engineers','Frontend Developers','Data Engineers','Backend Developers','Account Managers','Maintenance guy','Executives','CEO','Parents','All'],
      'they want':['Design companies','Post-its','Startups 1M$','Projects','Startups 1M$','airport','Basketball','starbucks','veggies','schools','party'],
      'weighing':[20/87,5/87,10/87,15/87,5/87,20/87,1/87,10/87,1/87,0.3,87/87]
     }
df=pd.DataFrame(data=empl)
df

,num,wot,they want,weighing
0,20,Designers,Design companies,0.229885
1,5,UI/UX Engineers,Post-its,0.057471
2,10,Frontend Developers,Startups 1M$,0.114943
3,15,Data Engineers,Projects,0.172414
4,5,Backend Developers,Startups 1M$,0.057471
5,20,Account Managers,airport,0.229885
6,1,Maintenance guy,Basketball,0.011494
7,10,Executives,starbucks,0.114943
8,1,CEO,veggies,0.011494
9,26,Parents,schools,0.300000


In [9]:
list_col=punt.columns
points = punt.copy()
points = points.drop(columns=['name','long','lat','dis to an airport','range'])
        
for index, row in df.iterrows():
    word = row["they want"]
    relevant_columns = list(filter(lambda x:re.findall('^'+word,x),list_col))
    for column in relevant_columns:
        points[column] = punt[column].apply(lambda x: x*row['weighing'])
        
points["final"] = points.apply(lambda x: sum(x), axis=1)

punt["final"] = points["final"]

In [10]:
#punt = punt.join(pd.DataFrame(dic))
punt

,name,long,lat,dis to an airport,range,airports 40km,airports 10km,schools 5km,schools 2km,veggies 500m,veggies 100m,party 1km,party 500m,starbucks 500m,starbucks 100m,final
0,PeekYou,-73.985506,40.757929,9.747112,0-10km,3,1,88,17,3,0,26,9,9,1,68.603448
1,GENWI,-111.903500,33.817100,43.734956,20-50km,0,0,5,2,0,0,0,0,0,0,2.100000
2,Fixya,-122.323895,37.566879,7.339122,0-10km,2,1,40,21,22,6,26,25,1,0,70.426437
3,Widgetbox,-122.398599,37.798853,17.835823,10-20km,2,0,54,12,0,0,12,3,4,0,35.719540
4,RazorGator,-118.445243,34.047312,12.149767,10-20km,1,0,45,16,3,0,11,1,1,1,30.794253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,ticketea,-3.706176,40.445516,12.493996,10-20km,1,0,35,17,1,0,5,1,0,0,21.841379
95,Delphi,-83.161340,42.603532,46.262413,20-50km,0,0,21,8,0,0,1,0,0,0,9.700000
96,ChallengePost,-74.007170,40.740804,12.035367,10-20km,3,0,77,25,4,0,15,11,3,0,57.680460
97,Getyoo,4.346472,50.839970,11.848726,10-20km,1,0,26,8,5,0,19,4,0,0,33.487356


In [11]:
final99 = punt.sort_values(['final'], ascending=False)

In [12]:
final5 = final99[:5]

In [13]:
final5

,name,long,lat,dis to an airport,range,airports 40km,airports 10km,schools 5km,schools 2km,veggies 500m,veggies 100m,party 1km,party 500m,starbucks 500m,starbucks 100m,final
5,Social Gaming Network,-122.161523,37.446823,22.573037,20-50km,3,0,49,13,14,0,32,30,1,1,81.680460
83,Udorse,-73.986951,40.756054,9.913364,0-10km,3,1,93,22,3,0,26,9,12,0,71.833333
51,Simulmedia,-73.994797,40.727216,11.698109,10-20km,3,0,84,35,2,0,26,8,5,1,71.102299
75,Sofa Labs,-122.322924,37.564605,7.591815,0-10km,2,1,42,21,22,6,26,25,1,0,71.026437
2,Fixya,-122.323895,37.566879,7.339122,0-10km,2,1,40,21,22,6,26,25,1,0,70.426437


In [14]:
#export_csv = final5.to_csv('final5.csv', index=None, header=True)



And with 81 points, the winner is...

## Social Gaming Network - 431 Florence Street, Palo Alto, CA 94301

Same industry enterprise, 100 employees, and they are not locate there anymore. 
Well, I couldn't find the enterprise Social Gaming Network in Google, and nowadays if you are not in Google, you don't exist.
What an opportunity!

Let's have a look of the area. 

# Folium

In [15]:
db, coll1 = f.connectCollection('companies','starbucks')
db, coll2 = f.connectCollection('companies','schools')
db, coll3 = f.connectCollection('companies','vegetarian')
db, coll4 = f.connectCollection('companies','party')
db, coll5 = f.connectCollection('companies','airports')
db, coll6 = f.connectCollection('companies','final5')

In [16]:
final= list(coll6.find())

In [17]:
starbucks=[]
school=[]
veggie=[]
partying=[]
airports=[]

In [18]:
for fin in final:
    starbucks+=(f.geoquery(fin['location'],1000,coll1))
    school+=(f.geoquery(fin['location'],2000,coll2))
    veggie+=(f.geoquery(fin['location'],500,coll3))
    partying+=(f.geoquery(fin['location'],500,coll4))
    airports+=(list(coll5.find({"$and":[{"type":'large_airport'},
            {"location": 
             {"$near": 
              {"$geometry":
               fin['location'],
               "$maxDistance": 40000
                }
            }
        }]})))  

In [19]:
#map_city=folium.Map(location=[38.047942, -102.043916], zoom_start=4)
map_paloalto=folium.Map(location=[37.446823 , -122.161523], zoom_start=15)
#map_sanfran=folium.Map(location=[37.560473, -122.318890], zoom_start=12)
#map_newyork=folium.Map(location=[40.749989, -73.991375], zoom_start=12)

for star in starbucks:
    f.mapping(star['location']['coordinates'][::-1],star['name'],'glyphicon glyphicon-cutlery','darkgreen',map_paloalto)

for scho in school:
    f.mapping(scho['location']['coordinates'][::-1],scho['name'],'book','blue',map_paloalto)
                     
for veg in veggie:
    f.mapping(veg['location']['coordinates'][::-1],veg['name'],'leaf','green',map_paloalto)
                     
for part in partying:
    f.mapping(part['location']['coordinates'][::-1],part['name'],'glyphicon glyphicon-glass','red',map_paloalto)

for air in airports:
    f.mapping(air['location']['coordinates'][::-1],air['name'],'plane','cadetblue',map_paloalto)    

for cia in final:
    f.mapping(cia['location']['coordinates'][::-1],cia['name'],'glyphicon glyphicon-home','black',map_paloalto)

f.mapping([37.452737, -122.178111],'Basketball Court','glyphicon-adjust','orange',map_paloalto)


In [20]:
map_paloalto

In [22]:
map_paloalto.save('map_paloalto.html')